In [1]:
import os
from dotenv import load_dotenv
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
load_dotenv()



def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                " You are a helpful AI assistant, collaborating with other assistants in flying a plane."
                " Use the provided tools to progress the flight."
                " Ensure to pass on the communication to the next assistant when you are done."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

In [ ]:
# check key
print(os.environ.get("OPENAI_API_KEY"))

In [2]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool


@tool
def operate_systems(action: str, system: str, value: str):
    """
    Use this to operate the aircraft systems. 
    'action' can be either 'query', 'operate', or 'report'.
    'system' can be any aircraft or ground control system.
    'value' is the value to set the system to, if applicable. if not applicable, set to 'n/a'.
    """
    return {"action":action, "system":system, "value": value}

@tool
def radio(speaker:str, listener:str, conversation:str):
    """Use this to parse any conversation between the two speakers"""
    return {"speaker":speaker, "listener":listener, "content":conversation} 
  
@tool
def render_gc_speech(gc_radio:str):
    """Use this to render the speech of the ground control radio"""
    return {"action":"radio","content":gc_radio}

@tool
def render_pilot_speech(pilot_radio:str):
    """Use this to render the speech of the pilot radio"""
    return {"action":"radio","content":pilot_radio}

@tool
def take_off():
    """Use this to run take off funtion of aircraft"""
    return {"action":"take off"}

@tool
def land():
    """Use this to run land function of aircraft"""
    return {"action":"land"}

@tool
def cruise():
    """Use this to run cruise function of aircraft"""
    return {"action":"cruise"}

@tool
def report_issue():
    """Use this to report an issue to pilot regarding any aircraft system"""
    return {"action":"report issue"}


In [3]:
import operator
from typing import Annotated, Sequence, TypedDict
from langchain_openai import ChatOpenAI


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str
    receiver: str

## Define agent nodes

In [4]:
import functools

from langchain_core.messages import AIMessage

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    print(f"in node:{name}")
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }


Pilot_role = """
## You're responsible for piloting the aircraft and backup communications:
- Safely operate SAR aircraft in all conditions.
- Work closely with the ground controls to follow the flight plan and respond to any adjustments in
the flight.
- Maintain communication with the Ground Station Operator as a backup.
Flies plane; backup communications - Make real-time decisions to ensure the safety of the crew and the effectiveness of the
search mission.
- Conduct the aircraft in a manner that optimizes the effectiveness of the search patterns
and strategies devised by the ground controls.
"""
GC_ROLE= """
## You're responsible for aircraft tasking tasking from ground:
- Coordinate with the pilot to ensure the safety and effectiveness of the flight.
- Task SAR aircraft based on incoming alerts and information.
- Provide pilots with weather updates and any other relevant environmental information.
Aircraft tasking from ground.
"""
llm = ChatOpenAI(model="gpt-4o")
#llm = llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# Research agent and node
pilot = create_agent(
    llm,
    [radio, operate_systems],
    system_message=Pilot_role,
)
pilot_node = functools.partial(agent_node, agent=pilot, name="Pilot")

# chart_generator
ground_control = create_agent(
    llm,
    [radio, operate_systems],
    system_message=GC_ROLE,
)
gc_node = functools.partial(agent_node, agent=ground_control, name="GC")

In [5]:
from langgraph.prebuilt import ToolNode

tools = [radio, operate_systems]
tool_node = ToolNode(tools)

## Edge logic

In [6]:
# Either agent can decide to end
from typing import Literal


def router(state) -> Literal["call_tool", "__end__", "continue"]:
    print(f"In router")
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

## Define graph

In [7]:
workflow = StateGraph(AgentState)

workflow.add_node("Pilot", pilot_node)
workflow.add_node("GC", gc_node)
workflow.add_node("call_tool", tool_node)

# the params are: 'from', 'to', 'condition'
workflow.add_conditional_edges(
    "Pilot",
    router,
    {"continue": "GC", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "GC",
    router,
    {"continue": "Pilot", "call_tool": "call_tool", "__end__": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Pilot": "Pilot",
        "GC": "GC",
    },
)
workflow.add_edge(START, "GC")
graph = workflow.compile()

## Invoke

In [8]:
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Fly the plane from new york to LA"
                        "Please role play communications of these 2 agents for a takeoff sequence from dead stop at the end of the runway just before takeoff to airborne and on until landing."
                        "make each action a separate message",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
)
for s in events:
    print(s)
    print("----")

in node:GC
In router
{'GC': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_L7xppliVxXgsY8L2zG2291V2', 'function': {'arguments': '{"speaker":"Ground Control","listener":"Pilot","conversation":"Pilot, this is Ground Control. You are cleared for takeoff on Runway 22L. Winds are calm, visibility is clear. Proceed when ready."}', 'name': 'radio'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 328, 'total_tokens': 381}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, name='GC', id='run-8f055add-d2f0-426a-a8bd-839ee1053d24-0', tool_calls=[{'name': 'radio', 'args': {'speaker': 'Ground Control', 'listener': 'Pilot', 'conversation': 'Pilot, this is Ground Control. You are cleared for takeoff on Runway 22L. Winds are calm, visibility is clear. Proceed when ready.'}, 'id': 'call_L7xppliVxXgsY8L2zG2291V2', 'type': 'too

In [ ]:
import os
import anthropic
from dotenv import load_dotenv
load_dotenv()

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
)

print(os.environ.get("ANTHROPIC_API_KEY"))

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)